# Azure AI Agent service - Many agents

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import json
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ListSortOrder
from azure.identity import DefaultAzureCredential
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI

In [2]:
load_dotenv()

True

In [3]:
sys.version

'3.13.4 (main, Jun  3 2025, 15:34:24) [Clang 17.0.0 (clang-1700.0.13.3)]'

## Creaate connection to AI Foundry Project & instantiate a project client

In [4]:
endpoint = os.getenv("PROJECT_ENDPOINT")
print(f"Using endpoint: {endpoint}")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

Using endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject


## Instantiate AGENT 1 (Q&A)

In [5]:


agent1 = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="Q&A",
    instructions="You are an AI agent.",
)
print(f"Created agent, agent ID: {agent1.id}")

Created agent, agent ID: asst_w42zCT2O5N5PjS3I6nom3upH


## Instantiate AGENT 2 (Translator)

In [6]:


agent2 = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="Translator",
    instructions="You are an AI agent that translates English into Spanish.",
)
print(f"Created agent, agent ID: {agent2.id}")

Created agent, agent ID: asst_6Gne1UjCvCuxPKbcKCpDptUK


## Instantiate AGENT 3 (Emojis creator)

In [7]:


agent3 = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="Emojis",
    instructions="You are an AI agent that add emojis to a text",
)
print(f"Created agent, agent ID: {agent3.id}")


Created agent, agent ID: asst_gZN9TiqXKBToioI3WONDdYLd


### Show all agents

In [8]:
print(agent1)
print(agent2)
print(agent3)


{'id': 'asst_w42zCT2O5N5PjS3I6nom3upH', 'object': 'assistant', 'created_at': 1749588027, 'name': 'Q&A', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are an AI agent.', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}
{'id': 'asst_6Gne1UjCvCuxPKbcKCpDptUK', 'object': 'assistant', 'created_at': 1749588028, 'name': 'Translator', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are an AI agent that translates English into Spanish.', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}
{'id': 'asst_gZN9TiqXKBToioI3WONDdYLd', 'object': 'assistant', 'created_at': 1749588028, 'name': 'Emojis', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are an AI agent that add emojis to a text', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}


### Create a thread for AGENT 1

In [9]:
    # [START create_thread]
    thread = project_client.threads.create()
    # [END create_thread]
    print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_gCcg2dBmlv9A8wfcCAiis1FF


### Create a message for AGENT 1

In [10]:

    # [START create_message]
    message = project_client.messages.create(thread_id=thread.id, role="user", content="Tell me about the 'Palacio de Bellas Artes' in Mexico City.")
    # [END create_message]
    print(f"Created message, message ID: {message.id}")

Created message, message ID: msg_cBlVwv1wV1bULwlpsJfZbuUf


### Run AGENT 1

In [11]:
    # [START create_run]
    run = project_client.runs.create(thread_id=thread.id, agent_id=agent1.id)
    # [END create_run]
    print(f"Created run, run ID: {run.id}")


Created run, run ID: run_atzPNjzxs67wlrUUuJk5PyZx


### Check status of run for AGENT 1

In [12]:
    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
        # [END create_run]
        print(f"Run status: {run.status}")

        if run.status == "failed":
            print(f"Run error: {run.last_error}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


### Show usage (tokens)

In [13]:
run.usage

{'prompt_tokens': 41, 'completion_tokens': 455, 'total_tokens': 496, 'prompt_token_details': {'cached_tokens': 0}}

### Show me the stack of messages so far

In [14]:
messages = project_client.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")





Message ID: msg_41WGjlEUxkynCfQNdbIGrJgJ, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': "Certainly! The **Palacio de Bellas Artes** (Palace of Fine Arts) is one of the most iconic and grand cultural landmarks in **Mexico City**, renowned for its stunning architecture and rich artistic significance.\n\n### History & Background\n- **Construction** began in 1904, initiated by President Porfirio Díaz, and was completed in 1934 due to various delays, including the Mexican Revolution.\n- Designed originally by architect **Adamo Boari** in an extravagant, neoclassical and art nouveau style, the interior was later finished by Mexican architect **Federico Mariscal** with strong **Art Deco** influences.\n\n### Architecture & Design\n- **Exterior:** White Italian Carrara marble, Art Nouveau and neoclassical designs.\n- **Interior:** Art Deco style, featuring locally sourced materials and motifs inspired by Aztec and Maya culture.\n- **Dome:** The multicolored glass and iro

### Show last message of AGENT 1

In [15]:
messages = list(project_client.messages.list(thread_id=thread.id))
if messages:
    last_message = messages[0]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': "Certainly! The **Palacio de Bellas Artes** (Palace of Fine Arts) is one of the most iconic and grand cultural landmarks in **Mexico City**, renowned for its stunning architecture and rich artistic significance.\n\n### History & Background\n- **Construction** began in 1904, initiated by President Porfirio Díaz, and was completed in 1934 due to various delays, including the Mexican Revolution.\n- Designed originally by architect **Adamo Boari** in an extravagant, neoclassical and art nouveau style, the interior was later finished by Mexican architect **Federico Mariscal** with strong **Art Deco** influences.\n\n### Architecture & Design\n- **Exterior:** White Italian Carrara marble, Art Nouveau and neoclassical designs.\n- **Interior:** Art Deco style, featuring locally sourced materials and motifs inspired by Aztec and Maya culture.\n- **Dome:** The multicolored glass and iron dome is a distinctive feature, glowing with reddish and orange to

### SHow bottom of stack of messages

In [16]:
if messages:
    last_message = messages[-1]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': "Tell me about the 'Palacio de Bellas Artes' in Mexico City.", 'annotations': []}}]


### Run AGENT 2

In [17]:
    # [START create_run]
    run = project_client.runs.create(thread_id=thread.id, agent_id=agent2.id)
    # [END create_run]
    print(f"Created run, run ID: {run.id}")

Created run, run ID: run_S4CFcN19VRIp6MxufyVwFIdA


### Check Status of the run for AGENT 2

In [18]:
    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
        # [END create_run]
        print(f"Run status: {run.status}")

        if run.status == "failed":
            print(f"Run error: {run.last_error}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


In [19]:
run.usage

{'prompt_tokens': 503, 'completion_tokens': 508, 'total_tokens': 1011, 'prompt_token_details': {'cached_tokens': 0}}

### SHow the stack of messages so far

In [20]:
messages = project_client.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")



Message ID: msg_yOeS04s03vic0NZTDaJsjEzm, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': 'Claro, aquí tienes la traducción al español:\n\nEl **Palacio de Bellas Artes** es uno de los recintos culturales más emblemáticos de la **Ciudad de México**, famoso por su impresionante arquitectura y su relevancia artística.\n\n### Historia y Antecedentes\n- La **construcción** comenzó en 1904 por orden del presidente Porfirio Díaz y se concluyó en 1934, debido a varios retrasos, incluyendo la Revolución Mexicana.\n- Su diseño original fue obra del arquitecto **Adamo Boari** en un estilo neoclásico y art nouveau; el interior fue terminado por el mexicano **Federico Mariscal**, quien incorporó elementos **art déco**.\n\n### Arquitectura y Diseño\n- **Exterior:** Mármol blanco de Carrara, con detalles neoclásicos y art nouveau.\n- **Interior:** Estilo art déco, con materiales nacionales y motivos inspirados en las culturas azteca y maya.\n- **Cúpula:** La colorida cúpula de h

### SHow last message of AGENT 2

In [21]:
messages = list(project_client.messages.list(thread_id=thread.id))
if messages:
    last_message = messages[0]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': 'Claro, aquí tienes la traducción al español:\n\nEl **Palacio de Bellas Artes** es uno de los recintos culturales más emblemáticos de la **Ciudad de México**, famoso por su impresionante arquitectura y su relevancia artística.\n\n### Historia y Antecedentes\n- La **construcción** comenzó en 1904 por orden del presidente Porfirio Díaz y se concluyó en 1934, debido a varios retrasos, incluyendo la Revolución Mexicana.\n- Su diseño original fue obra del arquitecto **Adamo Boari** en un estilo neoclásico y art nouveau; el interior fue terminado por el mexicano **Federico Mariscal**, quien incorporó elementos **art déco**.\n\n### Arquitectura y Diseño\n- **Exterior:** Mármol blanco de Carrara, con detalles neoclásicos y art nouveau.\n- **Interior:** Estilo art déco, con materiales nacionales y motivos inspirados en las culturas azteca y maya.\n- **Cúpula:** La colorida cúpula de hierro y cristal destaca por sus tonos anaranjados y dorados.\n\n###

### Run AGENT 3

In [22]:
    # [START create_run]
    run = project_client.runs.create(thread_id=thread.id, agent_id=agent3.id)
    # [END create_run]
    print(f"Created run, run ID: {run.id}")

Created run, run ID: run_5ZwYWWSUJplaFgHPFL6zpXEB


### Check status of run of AGENT 3

In [23]:
    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
        # [END create_run]
        print(f"Run status: {run.status}")

        if run.status == "failed":
            print(f"Run error: {run.last_error}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


In [24]:
run.usage

{'prompt_tokens': 1013, 'completion_tokens': 224, 'total_tokens': 1237, 'prompt_token_details': {'cached_tokens': 0}}

### Show the Stack of messages so far

In [25]:
messages = project_client.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")

Message ID: msg_FUojbarEEtM1ARE9VfdMgpdR, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': 'Absolutely! Here’s your text with emojis for a more vibrant touch:\n\nThe **Palacio de Bellas Artes** 🎭🏛️ in **Mexico City** 🇲🇽 is one of the most iconic landmarks in Mexico. It’s famous for its stunning architecture 🏰, blending **Art Nouveau** on the outside and **Art Deco** on the inside. Built with white Italian Carrara marble 🤍, its colorful domed roof stands out over Alameda Central park 🌳.\n\nThis cultural palace hosts concerts 🎶, operas 🎤, ballets 🩰, and art exhibitions 🎨, making it the country’s top center for the arts. Inside, you’ll find murals by legendary Mexican artists like **Diego Rivera** 👨\u200d🎨 and **David Alfaro Siqueiros**.\n\nThe beautiful stained glass stage curtain 🌈, made by Tiffany & Co., shows the Valley of Mexico with its famous volcanoes 🌋. The Palacio de Bellas Artes is not only a UNESCO World Heritage Site 🌐 but also a must-see if you’re visiti

### Print last message of AGENT 3, this shows all Agents work combined in final output!!

In [26]:
messages = list(project_client.messages.list(thread_id=thread.id))

last_message = messages[0]
print(f"Content: {last_message.content}")




Content: [{'type': 'text', 'text': {'value': 'Absolutely! Here’s your text with emojis for a more vibrant touch:\n\nThe **Palacio de Bellas Artes** 🎭🏛️ in **Mexico City** 🇲🇽 is one of the most iconic landmarks in Mexico. It’s famous for its stunning architecture 🏰, blending **Art Nouveau** on the outside and **Art Deco** on the inside. Built with white Italian Carrara marble 🤍, its colorful domed roof stands out over Alameda Central park 🌳.\n\nThis cultural palace hosts concerts 🎶, operas 🎤, ballets 🩰, and art exhibitions 🎨, making it the country’s top center for the arts. Inside, you’ll find murals by legendary Mexican artists like **Diego Rivera** 👨\u200d🎨 and **David Alfaro Siqueiros**.\n\nThe beautiful stained glass stage curtain 🌈, made by Tiffany & Co., shows the Valley of Mexico with its famous volcanoes 🌋. The Palacio de Bellas Artes is not only a UNESCO World Heritage Site 🌐 but also a must-see if you’re visiting Mexico City!', 'annotations': []}}]


## Post processing

In [27]:
# List all agents in the project
print("Listing all agents in the project:")
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

Listing all agents in the project:
Agent ID: asst_gZN9TiqXKBToioI3WONDdYLd, Name: Emojis, Model: gpt-4.1, Instructions: You are an AI agent that add emojis to a text
Agent ID: asst_6Gne1UjCvCuxPKbcKCpDptUK, Name: Translator, Model: gpt-4.1, Instructions: You are an AI agent that translates English into Spanish.
Agent ID: asst_w42zCT2O5N5PjS3I6nom3upH, Name: Q&A, Model: gpt-4.1, Instructions: You are an AI agent.


In [1]:
# recurse through all agents and delete them adding a test to stop if the agent is not found
for agent in project_client.list_agents():
    try:
        print(f"Deleting agent ID: {agent.id}, Name: {agent.name}")
        project_client.delete_agent(agent.id)
    except Exception as e:
        print(f"Error deleting agent ID: {agent.id}, Name: {agent.name}, Error: {e}")
        break

NameError: name 'project_client' is not defined

In [ ]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")